# Extraer datos de generación eléctrica
La ejecución de este código utiliza los datos de los informes mensuales de datos operativos eléctricos de la Compañía Administradora del Mercado Mayorista Eléctrico (CAMMESA) de Argentina, y los agrupa en un único archivo CSV y filtra duplicados. <br><br>
Dentro de los mismos consta un resumen mensual que incluye detalles de generación (MWh) por máquina, central, agente, región, tipo de máquina, tipo de fuente, tecnología y generación neta mensual.

In [1]:
# Cargar librerías
import zipfile
import numpy as np
import pandas as pd
from datetime import date, datetime
from dateutil.rrule import rrule, MONTHLY
from os import remove, path

In [2]:
# Definir fechas
sdate = date(2016, 1, 1)   # fecha de inicio (año, mes, día)
edate = date(2016, 5, 1)   # fecha de fin (año, mes, día)

In [3]:
# Definir valores a extraer
filePath = './CAMMESA'
fileName = 'BASE_INFORME_MENSUAL_'
sheetName = 'GENERACION'
headerRow = 22
columns = 'A:N'
columnNames = ['AÑO', 'MES', 'MAQUINA', 'CENTRAL', 'AGENTE', 'AGENTE_DESCRIPCION',
       'REGION', 'PAIS', 'TIPO_MAQUINA', 'FUENTE_GENERACION', 'TECNOLOGIA',
       'CATEGORIA_HIDRAULICA', 'CATEGORIA_REGION', 'GENERACION_NETA']

outFile = 'Generacion.csv' #nombre del archivo de destino

In [4]:
#Funciones de fecha
def pad_date(month_num):
    month = str(month_num)
    if len(month) == 1:
        month = '0' + month
    return month

def month_iter(start_month, start_year, end_month, end_year):
    start = datetime(start_year, start_month, 1)
    end = datetime(end_year, end_month, 1)

    return ((pad_date(d.month), pad_date(d.year)) for d in rrule(MONTHLY, dtstart=start, until=end))

In [ ]:
# Descomprimir datos
for (month, year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.zip'

    if path.isfile(inFile):
        print('Extracting ' + inFile)
        with zipfile.ZipFile(inFile) as zf:
            zf.extractall(filePath)
    else:
        print('ERROR: ' + inFile + " does not exist")

In [ ]:
# Borrar archivos zip
for (month, year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.zip'
    
    if path.isfile(inFile):
        print('Deleting ' + inFile)
        remove(inFile)
    else:
        print('ERROR: ' + inFile + " does not exist")

In [7]:
# Unificar datos en un DataFrame
out = pd.DataFrame()

for (month,year) in month_iter(sdate.month, sdate.year, edate.month, edate.year):
    nameComponents = [year, month]  
    inFile = filePath + '/' + fileName + '-'.join(nameComponents) + '.xlsx'
    
    if path.isfile(inFile):
        print('Concatenating ' + inFile)
        df = pd.read_excel(inFile, sheet_name = sheetName, header = headerRow, usecols = columns)
        df.columns = columnNames
        out = pd.concat([out, df])
    else:
        print('ERROR: ' + inFile + " does not exist")

Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-01.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-02.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-03.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-04.xlsx
Concatenating ./CAMMESA/BASE_INFORME_MENSUAL_2016-05.xlsx


In [8]:
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151950 entries, 0 to 32272
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   AÑO                   151950 non-null  int64         
 1   MES                   151950 non-null  datetime64[ns]
 2   MAQUINA               151950 non-null  object        
 3   CENTRAL               151950 non-null  object        
 4   AGENTE                151950 non-null  object        
 5   AGENTE_DESCRIPCION    151950 non-null  object        
 6   REGION                151950 non-null  object        
 7   PAIS                  151950 non-null  object        
 8   TIPO_MAQUINA          151950 non-null  object        
 9   FUENTE_GENERACION     151950 non-null  object        
 10  TECNOLOGIA            151950 non-null  object        
 11  CATEGORIA_HIDRAULICA  11120 non-null   object        
 12  CATEGORIA_REGION      151950 non-null  object        
 13  

In [9]:
#Drop duplicated rows
df = out.drop_duplicates(subset = columnNames, inplace = False)
df.reset_index(drop= True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34859 entries, 0 to 34858
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   AÑO                   34859 non-null  int64         
 1   MES                   34859 non-null  datetime64[ns]
 2   MAQUINA               34859 non-null  object        
 3   CENTRAL               34859 non-null  object        
 4   AGENTE                34859 non-null  object        
 5   AGENTE_DESCRIPCION    34859 non-null  object        
 6   REGION                34859 non-null  object        
 7   PAIS                  34859 non-null  object        
 8   TIPO_MAQUINA          34859 non-null  object        
 9   FUENTE_GENERACION     34859 non-null  object        
 10  TECNOLOGIA            34859 non-null  object        
 11  CATEGORIA_HIDRAULICA  4839 non-null   object        
 12  CATEGORIA_REGION      34859 non-null  object        
 13  GENERACION_NETA 

In [10]:
#Exporta la tabla
df.to_csv(outFile, index = False) 